# Decorators : closure, variable scope, nonlocal...

---------------------

source : "Fluent Python" by Luciano Ramalho

---------------------

## Decorators 101

Definition : a decorator is a callable that takes another function as argument (the decorated function). The decorator may perform some processing with the decorated function, and return it or replace it with another function or callable object.

Syntax and syntaxer

In [ ]:
def decorate(func):
    print('It\'s decorate!')
    return func

In [ ]:
@decorate
def target1():
    print('running target1()')
    
target1()

In [ ]:
def target2():
    print('running target2()')
    
decorate(target2) # Not decorate(target2())
target2()

-------------

Here is a decorator that replaces a function with a different one.

In [ ]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [ ]:
target()

In [ ]:
target

--------------------------------

## When Python executes decorators

![question pour un kahoot](assets/Kahoot_is_not_my_friend_anymore.png)

In [ ]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

=> They run right after the decorated function is defined. That is usually at import time.

---------------------

In [ ]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_orger(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    return order.total() * .07 if len(distinct_items) >= 10 else 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

--------------

Until now, our decorators don"t change the decorated function. To do so, we usually need to define an inner function and return it to replace the decorated function. Code that uses inner function almost always  depends on closures to operate correctly. And to understand closures, we need to look at how variables scopes work in Python.

----------------

## Variable Scope

In [ ]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

In [ ]:
b = 6
f1(3)

In [ ]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

The generate bytecode is different.

In [ ]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)
b

---------------

## Closures

A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not define there.

##### An average function using oop

In [ ]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [ ]:
avg = Averager()
avg(10)

In [ ]:
avg(11)

In [ ]:
avg(12)

----------------------------

##### Same, but using a higher-order function

In [ ]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [ ]:
avg = make_averager()
avg(10)

In [ ]:
avg(11)

In [ ]:
avg(12)

When invoked, make_averager returns an averager function object.

-----------------

Question is : where does this avg function find the series?

---------------

![Closures](assets/closures.svg)

Free variable : a variable that is not bound in the local scope.

In [ ]:
avg.__code__.co_varnames

In [ ]:
avg.__code__.co_freevars

In [ ]:
avg.__closure__

Each item here corresponds to a name in co_freevars

In [ ]:
avg.__closure__[0].cell_contents

------------------------

###### Conclusion about closure

A closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

Note that the only situation in which a function may need to deal with external variables that are nonglobal is when it's nested in another function.

-------------------------

##### So, let's optimize our code !

In [ ]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    
    return averager

In [ ]:
avg = make_averager()

In [ ]:
avg(10)

In [ ]:
avg(11)

In [ ]:
avg(12)

---------------------

## The non local Declaration

nonlocal : lets you flag a variable as a free variable even when it's assigned a new value within the function.

Nonlocal variables are used in nested functions whose local scope is not defined. This means that the variable can be neither in the local nor the global scope.

In [ ]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

In [ ]:
avg = make_averager()
avg(10)

In [ ]:
avg(11)

In [ ]:
avg(12)

In [ ]:
count

----------------

## Implementing a simple decorator

##### A decorator that output the running time of functions

In [ ]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [ ]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [ ]:
print('*' * 25, 'Calling snooze(.125)')
snooze(.125)
print('*' * 25, 'Calling factorial(6)')
factorial(6)

- clock gets factorial as its func argument
- It then creates and return the clocked function -> behind the scene, Python assigns it to factorial
- The factorial function is modified.

-----------------

In [ ]:
factorial.__name__

Factorial now holds a reference to the clocked function. From now on, each time factorial(n) is called, clocked(n) gets executed.

This is the typical behavior of a decorator. It replaces the decorated function with a new function that accepts the same arguments.

##### But I kinda liked the name of my function without the decorator...

No worries, functools has our back.

In [ ]:
import time
import functools

import time

def clock(func):
    @functools.wraps(func)
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [ ]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [ ]:
factorial.__name__

##### Another decorator in the standard library : functools.lru_cache

Memoization : an optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments.

LRU => Last Recently Used

In [ ]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [ ]:
print(fibonacci(6))

fibonacci(1) is called 8 times... => time wasted

In [ ]:
import functools

@functools.lru_cache() # can take parameters
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [ ]:
print(fibonacci(6))

## Stacked decorators

In [ ]:
@d1
@d2
def f():
    print('f')

is the same as :

In [ ]:
def f():
    print('f')
    
f = d1(d2(f))

## Parameterized Decorators

##### Activate or deactivate the register decorator

In [ ]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
    
print('registry ->', registry)
f1()

Let's try adding a parmeter

In [ ]:
registry = set() #Adding and removing from a set is faster

def register(func, active = True):
    print(f'running register(active={active}) -> decorate({func})')
    if active:
        registry.add(func)
    else:
        registry.discard(func)
    return func

@register(active=False)
def f1():
    print('running f1()')
    
@register(active=True)
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
    
print('registry ->', registry)
f1()
f2()
f3()

In [ ]:
registry = set()

def register(active = True):
    def decorate(func):
        print(f'running register(active={active}) -> decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register(active=True)
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
    
print('registry ->', registry)
f1()
f2()
f3()

Conceptually, the new register function is not a decorator but a decorator factory. When called, it returns the actual decorator that will be applied to the target function.

In [ ]:
register()(f3)
print('registry ->', registry)

--------------------------

But this parameterized decorator is simple, since it doesn't change the function. Parameterized decorators usually replace the decorated function, and their construction requires yet another level of nesting.

--------------------------

##### The clock decorator

In [ ]:
import time

DEFAULT_FMT = '[{elapsed:.8f}] {name}({arg_str}) -> {result}'

def clock(fmt=DEFAULT_FMT):               # parameterized decorator factory
    def decorate(func):                   # actual decorator
        def clocked(*args):               # wraps the decorated function
            t0 = time.time()
            result = func(*args)
            elapsed = time.time() - t0
            name = func.__name__
            arg_str = ', '.join(repr(arg) for arg in args)
            print(fmt.format(**locals()))
            return result
        return clocked
    return decorate

In [ ]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

In [ ]:
@clock()
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial(6)

In [ ]:
@clock('{name}({arg_str}) dt={elapsed:.3f}')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)